# 目的
FXのデータをダウンロードする。

In [1]:
import urllib.request
import json
import time
import datetime

In [2]:
def get_FX_from_gaitame(currencyPairCode="USDJPY"):
    """
    外為オンラインよりask/bidを取得する。
    
    ＜メソッドを作成するにあたって参考にしたHP＞
        http://qiita.com/chromabox/items/a1323225bae146c80bec
    
    ＜注意事項＞
        １秒ごとに更新されるが、このメソッドで１秒ごとに取得することはネチケット上やめるべき。
        最低でも２秒は空けたい。
    
    ＜入力＞
        currencyPairCode: 外貨ペア
    
    ＜出力＞
        nowtime: リクエストしたときの日時
        ask: 対象とする外貨ペアのask
        bid: 対象とする外貨ペアのbid
    """
    url = "http://www.gaitameonline.com/rateaj/getrate"
    nowtime = datetime.datetime.now()
    res = urllib.request.urlopen(url)
    quotes = json.loads(res.read().decode('utf-8'))["quotes"]
    for quo in quotes:
        if quo["currencyPairCode"] == currencyPairCode: 
            ask = float(quo["ask"])
            bid = float(quo["bid"])
            break
    return nowtime, ask, bid

def ToDataFrame(nts, fxs):
    """
    日時と値からデータフレームを生成する
    ＜入力＞
        nts: 日時（リスト）
        fxs: データ（[ask, bid]のリスト）
    """
    df = pd.DataFrame(fx, columns=["ask", "bid"])
    df.index = nowtimes
    return df

In [4]:
update_time = 300 # この時間ごとに更新 [sec]
update_count = 20 # この回数だけ取得したらリストを初期化する
savenow = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
savepath = "../data/fxdata_{0}.csv".format(savenow)

st = time.time()
nowtimes = []
fx = []
count = 0
isholiday = False
while True:
    try:
        nowtime, ask, bid = get_FX_from_gaitame()
        if isholiday == False:
            if nowtime.weekday() == 5:
                if nowtime.hour >= 6 and nowtime.minute > 1:
                    isholiday = True
                    sleep(60)
                    continue
        else:
            if nowtime.weekday() == 0 and nowtime.hour >= 6 and nowtime.minute > 1:
                isholiday = False
                continue
            else:
                print("holiday.")
                sleep(60)
                continue
        
        nowtimes.append(nowtime.strftime("%Y-%m-%d %H:%M:%S"))
        fx.append([ask, bid])
        time.sleep(5.0)
        if time.time() - st > update_time:
            st = time.time()
            df = ToDataFrame(nowtimes, fx)
            df.to_csv(savepath)
            count += 1
            if count >= update_count:
                nowtimes = []
                fx = []
                savenow = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                savepath = "../data/fxdata_{0}.csv".format(savenow)
                count = 0
#         print("Date:", nowtime.strftime("%Y-%m-%d %H:%M:%S"))
    except Exception as e:
        print(e)
        df = ToDataFrame(nowtimes, fx)
        df.to_csv(savepath)
        print("Wait for 1 minute...")
        time.sleep(60) # 失敗したときは１分ほど空ける
        nowtimes = []
        fx = []
        savenow = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        savepath = "../data/fxdata_{0}.csv".format(savenow)
        

[WinError 10054] 既存の接続はリモート ホストに強制的に切断されました。
Wait for 1 minute...
local variable 'ask' referenced before assignment
Wait for 1 minute...
local variable 'ask' referenced before assignment
Wait for 1 minute...
local variable 'ask' referenced before assignment
Wait for 1 minute...
local variable 'ask' referenced before assignment
Wait for 1 minute...
local variable 'ask' referenced before assignment
Wait for 1 minute...
local variable 'ask' referenced before assignment
Wait for 1 minute...
local variable 'ask' referenced before assignment
Wait for 1 minute...
local variable 'ask' referenced before assignment
Wait for 1 minute...
local variable 'ask' referenced before assignment
Wait for 1 minute...
local variable 'ask' referenced before assignment
Wait for 1 minute...
local variable 'ask' referenced before assignment
Wait for 1 minute...
local variable 'ask' referenced before assignment
Wait for 1 minute...
local variable 'ask' referenced before assignment
Wait for 1 minute...
local varia

KeyboardInterrupt: 